<a href="https://colab.research.google.com/github/moabe84/ACSFs_calculator/blob/main/ACSF_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the RDKit package

!pip install rdkit-pypi

In [16]:
# Import required libraries

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

In [17]:
# Define our classes and functions

class cutoff_func:
  def tanh_func(r, r_c):
    if r <= r_c:
      f = (np.tanh(1 - (r / r_c))**3) / (np.tanh(1)**3)
    else:
      f = 0
    return f

  def cos_func(r, r_c):
    if r <= r_c:
      f = 0.5 * (np.cos(np.pi*(r / r_c)) + 1)
    else:
      f = 0
    return f

  def exp_func(r, r_c):
    if r <= r_c:
      f = np.exp(1 - (1 / (1 - ((r / r_c)**2))))
    else:
      f = 0
    return f

class symm_func:
  def G_radial(r_ij, r_c, r_s, etha):
    g_rad = (np.exp(-etha*((r_ij - r_s)**2))) * cutoff_func.cos_func(r_ij, r_c)
    return g_rad

  def G_angular(r_ij, r_ik, r_jk, theta_ijk, r_c, etha, gamma, lammda):
    g_ang = (1 + (lammda * np.cos(theta_ijk)))**gamma * (np.exp(-etha*((r_ij**2 + r_ik**2 + r_jk**2)))) * ((cutoff_func.cos_func(r_ij, r_c)) * (cutoff_func.cos_func(r_ik, r_c)) * (cutoff_func.cos_func(r_jk, r_c)))
    return g_ang


class get_internal_coordinate:
  def bond_distance(x1, y1, z1, x2, y2, z2):
    bond = np.sqrt((float(x2) - float(x1))**2 + (float(y2) - float(y1))**2 + (float(z2) - float(z1))**2)
    return bond  # bond in Angstrom

  def bond_angle(x1, y1, z1, x2, y2, z2, x3, y3, z3):
    r_ij = (np.array([float(x2), float(y2), float(z2)])) - (np.array([float(x1), float(y1), float(z1)]))
    r_ik = (np.array([float(x3), float(y3), float(z3)])) - (np.array([float(x1), float(y1), float(z1)]))
    angle = np.arccos(np.dot(r_ij, r_ik) / (np.linalg.norm(r_ij) * np.linalg.norm(r_ik)))
    return angle * 57.2958  # angle in degree


def smiles_to_xyz(mol):
  mol = Chem.MolFromSmiles(mol)
  mol = Chem.AddHs(mol)
  AllChem.Compute2DCoords(mol)
  AllChem.EmbedMolecule(mol)
  AllChem.UFFOptimizeMolecule(mol)
  return Chem.MolToXYZFile(mol, 'mol')


In [ ]:
# Define our parameters

etha = 0.1
gamma = 1
lammda = -1
r_s = 0.0
r_c = 6.0

# Import SMILES

smiles = ['CNC(=O)c1nccc2cccn12']

# Calculate radial (two-body) ACSFs

for mol in smiles:
  print(f'Structure Formula: {CalcMolFormula(Chem.MolFromSmiles(mol))} \n')
  smiles_to_xyz(mol)
  data = open('mol', 'r')
  lines = list(csv.reader(data, delimiter=' ', skipinitialspace=True))
  num_atoms = int(lines[0][0])
  
  l = []
  for i in range(1, num_atoms+1):
    g = []
    for j in range(1, num_atoms+1):
      if i != j:
        dis = get_internal_coordinate.bond_distance(lines[i+1][1], lines[i+1][2], lines[i+1][3], lines[j+1][1], lines[j+1][2], lines[j+1][3])
        g_i = symm_func.G_radial(dis, r_c = r_c, r_s = r_s, etha = etha)
        g.append(g_i) 
        l.append(g_i)
    print(f'Sum of Gs for central atom {str(lines[i+1][0])}{i} = {np.sum(g)}')
  print('\n')
  g_mean = np.mean(l)
  g_std = np.std(l)
  g_max = np.max(l)
  g_min = np.min(l)
  print(f'Total Number of Atoms = {num_atoms} \n Total Number of Radial Gs = {len(l)} \n G(Average) = {g_mean} \n G(STD) = {g_std} \n G(MAX) = {g_max} \n G(MIN) = {g_mean}')

In [ ]:
# Calculate angular (three-body) ACSFs

for mol in smiles:
  print(f'Structure Formula: {CalcMolFormula(Chem.MolFromSmiles(mol))} \n')
  smiles_to_xyz(mol)
  data = open('mol', 'r')
  lines = list(csv.reader(data, delimiter=' ', skipinitialspace=True))
  num_atoms = int(lines[0][0])

  l = []
  for i in range(1, num_atoms+1):
    g = []
    for j in range(1, num_atoms+1):
      for k in range(1, num_atoms+1):
        if i != j and i != k and j != k:
          r_ij = get_internal_coordinate.bond_distance((lines[i+1][1]), (lines[i+1][2]), (lines[i+1][3]), (lines[j+1][1]), (lines[j+1][2]), (lines[j+1][3]))
          r_ik = get_internal_coordinate.bond_distance((lines[i+1][1]), (lines[i+1][2]), (lines[i+1][3]), (lines[k+1][1]), (lines[k+1][2]), (lines[k+1][3]))
          r_jk = get_internal_coordinate.bond_distance((lines[j+1][1]), (lines[j+1][2]), (lines[j+1][3]), (lines[k+1][1]), (lines[k+1][2]), (lines[k+1][3]))
          theta_ijk = get_internal_coordinate.bond_angle((lines[i+1][1]), (lines[i+1][2]), (lines[i+1][3]), (lines[j+1][1]), (lines[j+1][2]), (lines[j+1][3]), (lines[k+1][1]), (lines[k+1][2]), (lines[k+1][3]))
          g_i = symm_func.G_angular(r_ij, r_ik, r_jk, theta_ijk, r_c = r_c, etha = etha, gamma = gamma, lammda = lammda)
          g.append(g_i)
          l.append(g_i)
    print(f'Sum of Gs for central atom {str(lines[i+1][0])}{i} = {np.sum(g)}')
  print('\n')  
  g_mean = np.mean(l)
  g_std = np.std(l)
  g_max = np.max(l)
  g_min = np.min(l)
  print(f'Total Number of Atoms = {num_atoms} \n Total Number of Angular Gs = {len(l)} \n G(Average) = {g_mean} \n G(STD) = {g_std} \n G(MAX) = {g_max} \n G(MIN) = {g_mean}')  